# 🚀 Пример использования окружения F-4C Phantom II

Этот notebook демонстрирует работу с линейной продольной моделью истребителя F-4C Phantom II в библиотеке TensorAeroSpace.

## 📋 Что мы изучим:
- Настройка параметров симуляции
- Создание и инициализация окружения
- Выполнение шагов симуляции
- Анализ результатов


## 📦 Импорт необходимых библиотек

Загружаем все необходимые модули для работы с окружением F-4C:

In [1]:
# Основные библиотеки
import gymnasium as gym
import numpy as np
from tqdm import tqdm

# Модули TensorAeroSpace
from tensoraerospace.envs import LinearLongitudinalF4C
from tensoraerospace.utils import generate_time_period, convert_tp_to_sec_tp
from tensoraerospace.signals.standart import unit_step

## ⚙️ Настройка параметров симуляции

Определяем основные параметры для моделирования полета F-4C:

In [2]:
# Параметры временной дискретизации
dt = 0.01  # Шаг дискретизации (секунды)
tp = generate_time_period(tn=20, dt=dt)  # Временной период (20 секунд)
tps = convert_tp_to_sec_tp(tp, dt=dt)    # Преобразование в секунды

# Параметры симуляции
number_time_steps = len(tp)  # Общее количество временных шагов

# Создание эталонного сигнала (ступенчатый сигнал 5° на 10-й секунде)
reference_signals = np.reshape(
    unit_step(degree=5, tp=tp, time_step=10, output_rad=True), 
    [1, -1]
)

print(f'📊 Настройки симуляции:')
print(f'   • Время симуляции: {tp[-1]:.1f} секунд')
print(f'   • Шаг дискретизации: {dt} секунд')
print(f'   • Количество шагов: {number_time_steps}')
print(f'   • Эталонный сигнал: ступенька {5}° на {10}-й секунде')

📊 Настройки симуляции:
   • Время симуляции: 2002.0 секунд
   • Шаг дискретизации: 0.01 секунд
   • Количество шагов: 2002
   • Эталонный сигнал: ступенька 5° на 10-й секунде


## 🛩️ Создание и инициализация окружения F-4C

Создаем окружение с заданными параметрами и выполняем сброс в начальное состояние:

In [3]:
# Создание окружения F-4C Phantom II
env = gym.make(
    'LinearLongitudinalF4C-v0',
    number_time_steps=number_time_steps,
    initial_state=[[0], [0], [0], [0]],  # [θ, q, α, V] - начальное состояние
    reference_signal=reference_signals
)

# Сброс окружения в начальное состояние
initial_observation, info = env.reset()

print(f'🎯 Начальное состояние самолета:')
print(f'   • Тангаж θ: {initial_observation[0][0]:.3f} рад')
print(f'   • Угловая скорость q: {initial_observation[1][0]:.3f} рад/с')
print(f'   • Угол атаки α: {initial_observation[2][0]:.3f} рад')
print(f'   • Скорость V: {initial_observation[3][0]:.3f} м/с')

# Возвращаем результат для отображения
(initial_observation, info)

🎯 Начальное состояние самолета:
   • Тангаж θ: 0.000 рад
   • Угловая скорость q: 0.000 рад/с
   • Угол атаки α: 0.000 рад
   • Скорость V: 0.000 м/с


(array([[0.],
        [0.],
        [0.],
        [0.]], dtype=float32),
 {})

## 🎮 Выполнение шага симуляции

Подаем управляющий сигнал на руль высоты и наблюдаем реакцию самолета:

In [5]:
# Подача управляющего воздействия (отклонение руля высоты)
elevator_input = np.array([[1]])  # 1 единица отклонения руля высоты

# Выполнение одного шага симуляции
observation, reward, terminated, truncated, info = env.step(elevator_input)

print(f'🎯 Состояние после управляющего воздействия:')
print(f'   • Тангаж θ: {observation[0][0]:.6f} рад')
print(f'   • Угловая скорость q: {observation[1][0]:.6f} рад/с')
print(f'   • Угол атаки α: {observation[2][0]:.6f} рад')
print(f'   • Скорость V: {observation[3][0]:.6f} м/с')
print(f'💰 Награда: {reward:.8f}')
print(f'🏁 Завершено: {terminated}')
print(f'⏰ Прервано: {truncated}')

🎯 Состояние после управляющего воздействия:
   • Тангаж θ: 0.000027 рад
   • Угловая скорость q: -0.000592 рад/с
   • Угол атаки α: -0.000001 рад
   • Скорость V: -0.000000 м/с
💰 Награда: 0.00002700
🏁 Завершено: False
⏰ Прервано: False


## 📊 Анализ внутреннего состояния модели

Исследуем внутренние переменные модели для лучшего понимания динамики:

In [6]:
# Просмотр истории управляющих воздействий
input_history = env.unwrapped.model.store_input

print(f'📈 История управляющих воздействий:')
print(f'   • Размерность: {input_history.shape}')
print(f'   • Первые 5 значений: {input_history[0][:5]}')
print(f'   • Последние 5 значений: {input_history[0][-5:]}')

# Возвращаем для отображения
input_history

📈 История управляющих воздействий:
   • Размерность: (1, 2002)
   • Первые 5 значений: [1. 0. 0. 0. 0.]
   • Последние 5 значений: [0. 0. 0. 0. 0.]


array([[1., 0., 0., ..., 0., 0., 0.]])

In [7]:
# Анализ эталонного сигнала
current_reference = env.unwrapped.reference_signal[0][1]

print(f'🎯 Эталонный сигнал:')
print(f'   • Текущее значение: {current_reference:.3f} рад')
print(f'   • Размерность сигнала: {env.unwrapped.reference_signal.shape}')

# Возвращаем текущее значение
current_reference

🎯 Эталонный сигнал:
   • Текущее значение: 0.000 рад
   • Размерность сигнала: (1, 2002)


0.0

## 🏆 Анализ системы вознаграждений

Изучаем значение награды, полученной за выполненное действие:

In [8]:
# Детальный анализ награды
print(f'💰 Анализ награды:')
print(f'   • Значение: {reward:.2e}')
print(f'   • Тип данных: {type(reward).__name__}')
print(f'   • Знак: {"+" if reward >= 0 else "-"}')

if reward > 0:
    print(f'   ✅ Положительная награда - хорошее действие!')
elif reward < 0:
    print(f'   ❌ Отрицательная награда - нужно улучшить управление')
else:
    print(f'   ⚪ Нулевая награда - нейтральное действие')

# Возвращаем значение награды
reward

💰 Анализ награды:
   • Значение: 2.70e-05
   • Тип данных: float
   • Знак: +
   ✅ Положительная награда - хорошее действие!


2.6995467013112974e-05

## 🎉 Заключение

В этом примере мы успешно:

✅ **Создали окружение F-4C Phantom II** с правильными параметрами
✅ **Настроили временные параметры** симуляции
✅ **Сгенерировали эталонный сигнал** для отслеживания
✅ **Выполнили управляющие воздействия** и получили отклик системы
✅ **Проанализировали результаты** симуляции

### 🚀 Следующие шаги:
- Попробуйте различные управляющие воздействия
- Измените параметры эталонного сигнала
- Реализуйте контроллер для автоматического управления
- Визуализируйте результаты симуляции

---

*Этот notebook демонстрирует базовые возможности TensorAeroSpace для моделирования авиационных систем.*

In [9]:
# 🧹 Очистка ресурсов
env.close()
print("🔒 Окружение закрыто. Ресурсы освобождены.")

🔒 Окружение закрыто. Ресурсы освобождены.
